In [10]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from implicit.evaluation import train_test_split
from cv_py import CrossValidation
from scipy import sparse

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [73]:
dl = DataLoader()

In [74]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [75]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [5]:
# initial split
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [8]:
cv = CrossValidation(5)
validate, calibrate = cv.split_k_fold(train, 22)
validate_t, calibrate_t = cv.split_k_fold(train_t, 22)

In [9]:
factors = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
iterations = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

OEM1:

In [246]:
space_iALS = {'factors' : factors, 'regularization' : [90], 'alpha' : [0.3], 'iterations' : iterations}
hyper_ials = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')
hyper_ials.to_csv('curves_ials.csv')

In [ ]:
space_eALS = {'factors' : factors, 'regularization' : [230], 'alpha' : [0.1], 'w0' : [60000], 'iterations' : iterations}
hyper_eals = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_eALS, model_class='eALS')
hyper_eals.to_csv('curves_eals.csv')

In [ ]:
space_BPR = {'factors' : factors, 'regularization' : [0.03], 'learning_rate' : [0.005], 'iterations' : iterations}
hyper_bpr = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')
hyper_bpr.to_csv('curves_bpr.csv')

In [ ]:
space_LMF = {'factors' : factors, 'regularization' : [40], 'learning_rate' : [1.2], 'iterations' : iterations, 'neg_prop': [50]}
hyper_lmf = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')
hyper_lmf.to_csv('curves_lmf.csv')

In [ ]:
interactions_csr = sparse.load_npz('interactions_csr.npz')
weights_csr = sparse.load_npz('weights_csr.npz')
itemf = sparse.load_npz('itemf.npz')
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)
val_w, cal_w = cv.split_k_fold(train_w, 22)
val_i, cal_i = cv.split_k_fold(train_i, 22)

In [ ]:
space_FM_warp = {'factors' : factors, 'regularization' : [0.0005], 'learning_rate' : [0.05], 'iterations' : iterations, 'loss' : ['warp'], 'max_sampled' : [100]}
hyper_fm = cv.hyperp_tuning(test=val_w, train=cal_w, exclude=test_w, seed=22, param_space=space_FM_warp, model_class='FM', item_features=itemf)
hyper_fm.to_csv('curves_fm.csv')

OEM2:

In [247]:
space_iALS_t = {'factors' : factors, 'regularization' : [90], 'alpha' : [1.3], 'iterations' : iterations}
hyper_ials_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')
hyper_ials_t.to_csv('curves_ials_t.csv')

In [ ]:
space_eALS_t = {'factors' : factors, 'regularization' : [10], 'alpha' : [0.1], 'w0' : [2000], 'iterations' : iterations}
hyper_eals_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')
hyper_eals_t.to_csv('curves_eals.csv')

In [249]:
space_BPR_t = {'factors' : factors, 'regularization' : [0.04], 'learning_rate' : [0.03], 'iterations' : iterations}
hyper_bpr_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_BPR_t, model_class='BPR')
hyper_bpr_t.to_csv('curves_bpr_t.csv')

In [250]:
space_LMF_t = {'factors' : factors, 'regularization' : [20], 'learning_rate' : [1.2], 'iterations' : iterations, 'neg_prop': [0]}
hyper_lmf_t = cv.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_LMF_t, model_class='LMF')
hyper_lmf_t.to_csv('curves_lmf_t.csv')

In [ ]:
interactions_csr_t = sparse.load_npz('interactions_csr_t.npz')
weights_csr_t = sparse.load_npz('weights_csr_t.npz')
userf_t = sparse.load_npz('userf_t.npz')
train_i_t, test_i_t = train_test_split(interactions_csr, 0.8, 22)
train_w_t, test_w_t = train_test_split(weights_csr, 0.8, 22)
val_w_t, cal_w_t = cv.split_k_fold(train_w, 22)
val_i_t, cal_i_t = cv.split_k_fold(train_i, 22)

In [ ]:
space_FM_warp_t = {'factors' : factors, 'regularization' : [0.0001], 'learning_rate' : [0.001], 'iterations' : iterations, 'loss' : ['warp'], 'max_sampled' : [5]}
hyper_fm_t = cv.hyperp_tuning(test=val_w_t, train=cal_w_t, exclude=test_w_t, seed=22, param_space=space_FM_warp_t, model_class='FM', user_features=userf_t)
hyper_fm_t.to_csv('curves_fm_t.csv')